In [ ]:
!pip install gradio
!pip install datasets transformers[torch] seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 527.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing 

In [ ]:
import gradio as gr
import joblib
import torch
from transformers import AutoModelForTokenClassification,AutoModelForSequenceClassification, Trainer, AutoTokenizer, DistilBertForTokenClassification, TrainingArguments, DataCollatorWithPadding, AutoConfig

Importing Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Loading trained model
output_dir = "/content/drive/My Drive/SmishingData/Model/GhContext_model"
gh_model = AutoModelForSequenceClassification.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')


# Loading model configuration
config = AutoConfig.from_pretrained(output_dir)

#Label mappings
label2id = {'ham': 0, 'smish': 1}
id2label = {0: 'ham', 1: 'smish'}

# Updating the configuration with the new label mappings
config.label2id = label2id
config.id2label = id2label

# Load the model with the updated configuration
gh_model = AutoModelForTokenClassification.from_pretrained(output_dir, config=config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def predict(text):
    try:
        # Prepare the text for the model
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)

        # Get predictions
        with torch.no_grad():
            outputs = gh_model(**inputs)

        # Logits for each token
        logits = outputs.logits
        # Getting probabilities for each token
        probabilities = torch.nn.functional.softmax(logits, dim=-1)

        # Extracting the predicted class
        predicted_classes = torch.argmax(probabilities, dim=-1)
        predicted_confidences = torch.max(probabilities, dim=-1).values

        # Format the output message
        predicted_labels = ['smish' if pred == 0 else 'ham' for pred in predicted_classes[0].tolist()]
        confidence_scores = [conf.item() * 100 for conf in predicted_confidences[0]]
        predicted_label = predicted_labels[0]
        confidence_score = confidence_scores[0]
        result_message = f"Predicted Class: {predicted_label} with {confidence_score:.2f}% confidence."
        return result_message

    except Exception as e:
        # Handle exceptions and provide feedback
        error_message = f"An error occurred: {e}"
        return error_message


In [ ]:
interface = gr.Interface(
    fn=predict,
    inputs=gr.components.Textbox(label="Input Box",lines=2, placeholder="Enter a text message"),
    outputs=gr.components.Textbox(label="Prediction"),
    title="Smishing Detection",
    description="Not sure whether a text you've gotten is harmless or a scam. Enter it here and lets check it for you.",
    theme="default"
)

In [ ]:
if __name__ == "__main__":
    interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://666001b0a90210083e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
